In [6]:
import dask.array as da
import rasterio
import numpy as np
from dask import delayed, compute
import os

def create_large_raster_condition(input_raster1_path, input_raster2_path, input_raster3_path, output_raster_path, condition_function):
    """
    Create a large raster based on the condition of three other rasters.

    Parameters:
    - input_raster1_path: str, path to the first input raster file.
    - input_raster2_path: str, path to the second input raster file.
    - input_raster3_path: str, path to the third input raster file.
    - output_raster_path: str, path to the output raster file.
    - condition_function: function, a function that takes three arrays as input and returns an array.
    """

    # Open the input rasters
    with rasterio.open(input_raster1_path) as src1, rasterio.open(input_raster2_path) as src2, rasterio.open(input_raster3_path) as src3:
        # Check if the input rasters have the same shape and transform
        assert src1.shape == src2.shape == src3.shape, "Input rasters must have the same shape."
        assert src1.transform == src2.transform == src3.transform, "Input rasters must have the same transform."

        # Get metadata from one of the input rasters
        meta = src1.meta.copy()

        # Prepare Dask arrays for input rasters
        input_raster1 = da.from_array(src1.read(1), chunks=(1000, 1000))
        input_raster2 = da.from_array(src2.read(1), chunks=(1000, 1000))
        input_raster3 = da.from_array(src3.read(1), chunks=(1000, 1000))

        # Apply the condition function using Dask
        output_raster = condition_function(input_raster1, input_raster2, input_raster3)

        # Compute the result
        output_raster = output_raster.compute()

        # Update metadata for the output raster
        meta.update(dtype=rasterio.uint8, count=1, compress='deflate')

        # Write the output raster to disk
        with rasterio.open(output_raster_path, 'w', **meta) as dst:
            dst.write(output_raster, 1)

# Example condition function
def example_condition(raster1, raster2, raster3):
    """
    Example condition function. Modify this function to implement your condition.

    Parameters:
    - raster1: dask.array, first input raster.
    - raster2: dask.array, second input raster.
    - raster3: dask.array, third input raster.

    Returns:
    - dask.array, output raster based on the condition.
    """
    condition1 = (raster1 > 0) & (raster1 <= raster2)
    condition2 = (raster1 > 0) & (raster1 > raster2) & (raster1 <= raster3)
    condition3 = (raster1 > 0) & (raster1 > raster3)

    output = da.where(condition1, 1, da.where(condition2, 2, da.where(condition3, 3, 0)))

    return output


In [ ]:
# Example usage
input_raster1_path = 'path/to/input_raster1.tif'
input_raster2_path = 'path/to/input_raster2.tif'
input_raster3_path = 'path/to/input_raster3.tif'
output_raster_path = 'path/to/output_raster.tif'

create_large_raster_condition(input_raster1_path, input_raster2_path, input_raster3_path, output_raster_path, example_condition)



In [5]:
# Example usage
# Path to data folders
project_f = r'L:\f02_data\wildfires\spatial_data'
dsos_100m_folder =  os.path.join(project_f, 'output', 'dsos_100m')
deos_100m_folder =  os.path.join(project_f, 'output', 'deos_100m')
outFIREYEARDOY_folder =  os.path.join(project_f, 'output', 'FIRE_YEARDOY')

yr = 2019

input_raster1_path = os.path.join(outFIREYEARDOY_folder, f'FIRE_YEARDOY_{yr}.tif')
input_raster2_path = os.path.join(dsos_100m_folder, f'dsos{yr}_s1_100m_nearest_zstd.tif')
input_raster3_path = os.path.join(deos_100m_folder, f'deos{yr}_s1_100m_nearest_zstd.tif')

testfolder = r'L:\f02_data\wildfires\spatial_data\devdata'
output_raster_path = os.path.join(testfolder, f'GSMoment_{yr}_v2.tif')

create_large_raster_condition(input_raster1_path, input_raster2_path, input_raster3_path, output_raster_path, example_condition)


In [7]:
# Example usage
# Path to data folders
project_f = r'L:\f02_data\wildfires\spatial_data'
dsos_100m_folder =  os.path.join(project_f, 'output', 'dsos_100m')
deos_100m_folder =  os.path.join(project_f, 'output', 'deos_100m')
outFIREYEARDOY_folder =  os.path.join(project_f, 'output', 'FIRE_YEARDOY')

yr = 2019

input_raster1_path = os.path.join(outFIREYEARDOY_folder, f'FIRE_YEARDOY_{yr}.tif')
input_raster2_path = os.path.join(dsos_100m_folder, f'dsos{yr}_s1_100m_nearest_zstd.tif')
input_raster3_path = os.path.join(deos_100m_folder, f'deos{yr}_s1_100m_nearest_zstd.tif')

testfolder = r'L:\f02_data\wildfires\spatial_data\devdata'
output_raster_path = os.path.join(testfolder, f'GSMoment_{yr}_v3.tif')

create_large_raster_condition(input_raster1_path, input_raster2_path, input_raster3_path, output_raster_path, example_condition)
